In [1]:
import numpy as np
import cv2
import math
import matplotlib.pyplot as plt

In [51]:
## In case we want histogram from scratch as well, we can use assignment 1 histogram func
"""
def histogram(img):
    l, m = np.shape(img)
    h = np.array([np.arange(0,256), np.zeros(256)], dtype=int)
    for k in range(l):
        for i in range(m):
            pix=img[k, i]
            h[1, pix] += 1
    return h 
"""

In [7]:
def contrast_stretching(location, min_value = 0, max_value = 255):
    
    def check_gray(img_path):
        img = cv2.imread(img_path)
        if img.ndim == 2:
            return True
        elif img.ndim == 3:
            w, h, _ = img.shape
            for i in range(w):
                for j in range(h):
                    r, g, b = img[i,j,:]
                    if r != g != b: 
                        return False
        return True

    def cs_2D(img, mino, maxo):

        hist,_ = np.histogram(img.flatten(),256,[0,256])
        hist =  hist.reshape(1,256)
        y = np.zeros((1,256))

        for i in range(256):
            if hist[0,i] == 0:
                y[0,i] = 0
            else:
                y[0,i] = i
        mini = np.min(img)
        maxi = np.max(img)
        stretch = np.round((((maxo - mino)/(maxi - mini)*(y - mini)) + mino))
        stretch[stretch < 0] = 0
        stretch[stretch > 255] = 255
        for i in range(img.shape[0]):
                for j in range(img.shape[1]):
                    k = img[i,j]
                    img[i,j] = stretch[0,k]
        return img
    
    if check_gray(location):
        image = cv2.imread(location, 0)
        output = np.zeros_like(image)
        output = cs_2D(image, min_value, max_value)
    else:
        image = cv2.imread(location,1)
        output = np.zeros_like(image)
        for i in range(3):
            output[:,:,i] = cs_2D(image[:,:,i], min_value, max_value)
        
    return output

In [8]:
def histogram_equalization(location):
    
    def check_gray(img_path):
        img = cv2.imread(img_path)
        if img.ndim == 2:
            return True
        elif img.ndim == 3:
            w, h, _ = img.shape
            for i in range(w):
                for j in range(h):
                    r, g, b = img[i,j,:]
                    if r != g != b: 
                        return False
        return True
    
    def cum_sum(a):
        result = np.zeros_like(a)
        result[0,0] = a[0,0]
        for i in range(255):
            result[0,i+1]= result[0,i]+ a[0,i+1] 
        """"    
        a = iter(a)
        b = [next(a)]
        for i in a:
            b.append(b[-1] + i)
        b = np.array(b)
        """
        return result
    
    def equi_2D(img):
        copy = img.copy()
        hist,_ = np.histogram(img.flatten(),256,[0,256])
        hist =  hist.reshape(1,256)
        
        cdf = cum_sum(hist)
        cdf_norm = np.round((cdf * 255)/np.size(img))
        #cdf_norm = cdf_norm.astype('uint8')
        for i in range(img.shape[0]):
            for j in range(img.shape[1]):
                k = img[i,j]
                img[i,j] = cdf_norm[0,k]
        """"
        print(cdf_norm)      
        plt.figure()
        plt.hist(copy.flatten(),256,[0,256], color = 'b')
        plt.hist(img.flatten(),256,[0,256], color = 'r')
        plt.plot(cdf_norm, color = 'g')
        plt.xlim([0,256])
        plt.legend(('original','histogram_equal','cdf'), loc = 'upper left')
        plt.show()
        """
        return img
    
    

    if check_gray(location):
        image = cv2.imread(location,0)
        output = equi_2D(image)
        
    else:
        image = cv2.imread(location,1)
        
        output = np.zeros_like(image)
        for i in range(3):
            output[:,:,i] = equi_2D(image[:,:,i])
            
   
    return output

In [12]:
X  = "Lena.png"
original = cv2.imread(X)
print(original.shape)
hist,bins = np.histogram(original.flatten(),256,[0,256])
#print(hist,hist.shape,hist[0])
op =histogram_equalization(X)
g_op = gamma_correction(X,4)

ahe_op =adaptive_histogram(X, 50, np.array([[8], [8]]) )
st_op = histogram_stretching(X)
cs_op = contrast_stretching(X, 50,200)
print(np.min(cs_op),np.max(cs_op))

cv2.imshow("Input_image", cv2.imread(X))
cv2.imshow("Hist equal", op)
cv2.imshow("Contrast",cs_op)
cv2.imshow("stretch",st_op)
cv2.imshow("gamma",g_op)
cv2.imshow("CLAHE", ahe_op)
cv2.waitKey(0)
cv2.destroyAllWindows()

(512, 512, 3)
50 200


In [ ]:
# Trial 1 : Not completed

def adaptive_histogram(location, cliplimit = 40, tileGridSize = np.array([[8], [8]])):
    image = cv2.imread(location,0)
    
    if (Xsize > image.shape[0] or Ysize > image.shape[1]):
        return "tileGridSize cant be greater than image size"
    """
    if ( image.shape[0] % Xsize ):
        return "x-resolution no multiple of tile_x"
    if ( image.shape[1] % Ysize ):
        return "y-resolution no multiple of tile_y"
    """
    
    Xsize = tileGridSize[0,0]
    Ysize = tileGridSize[1,0]
    X_nt = np.fix(image.shape[0] / Xsize)    # no. of tiles in X direction
    Y_nt = np.fix(image.shape[1] / Ysize)    # no. of tiles in Y direction
    num = Xsize * Ysize
    op_image = np.zeros_like(image)
    

    def cum_sum(a):
        result = np.zeros_like(a)
        result[0] = a[0]
        for i in range(255):
            result[i+1]= result[i]+ a[i+1] 
        return result
    
    def clip_2D(hist_img, Cliplimit):
        Excess_pix = 0    
        
        # Calculate the total number of excess pixels.
        for k in range (hist_img.size()):
            excess = hist_img(k) - ClipLimit
                if excess > 0:
                    Excess_pix += excess
            
        # clip histogram and redistribute excess pixels in each bin 
        BinIncr = Excess_pix / hist_img.size()  #avg bin increment
        upper_limit = ClipLimit - BinIncr   # bins larger than this set to cliplimit 
        for i in range(hist_img.size()):
            if hist_img[i] > Cliplimit:
                hist_img[i] = Cliplimit
            else:
                if hist_img[i] > upper_limit :
                    Excess_pix += upper_limit - hist_img[i]
                    hist_img[i] = Cliplimit
                else:
                    Excess_pix -= BinIncr
                    hist_img[i] += BinIncr
                    
        # if 0 < excess_pixcels < BinIncr
        if Excess_pix > 0:
            for i in range(hist_img.size()):
                if hist_img[i] != Cliplimit:
                    hist_img[i] += 1
                    Excess_pix -= 1
                    if Excess_pix < 1:
                        break
        return hist_img
    
    def interpolate_2D(subBin, LU,LB, RU, RB):
        subImage = np.zeros_like(subBin)
        total = XSize * YSize
        for i in range (XSize):
            inv_I = XSize - i
            for j in range (YSize):
                inv_J = YSize - j
                val = subBin[i,j]
                subImage[i,j] = np.min(np.round((inv_I*(inv_J*LU(val) + j*RU(val)) + i*(inv_J*LB(val) + j*RB(val)))/total),255)
        return subImage
                
    cdf_norm = np.zeros(X_nt,Y_nt, 256)
    for x in range(X_nt):
        for y in range(Y_nt):
            hist, _ = np.histogram(image[x*Xsize: (x+1)*Xsize, y*Ysize: (y+1)*Ysize].flatten(),256,[0,256])
            hist = clip_2D(hist, cliplimit)
            cdf = cum_sum[hist]
            cdf = np.round((cdf * 255)/(Xsize*Ysize))
            cdf_norm[x,y,:]= cdf_norm[x, y, cdf]
            
    xI = 1
    for i in range (X_nt):
        if i == 0 or i == X_nt-1:          # top-bottom row
            subX = XSize/2
            xU = i
            xB = i
        else:
            subX = XSize
            xU = i - 1
            xB = i 
        yI = 1
        for j  in range(Y_nt):
            if j == 0 or j == Y_nt-1:      # left-right column
                subY = YSize/2      
                yL = j
                yR = j
            else:
                subY = YSize
                yL = j - 1
                yR = j
            
            UL = Map(xU,yL,:)
            UR = Map(xU,yR,:)
            BL = Map(xB,yL,:)
            BR = Map(xB,yR,:)
        

In [9]:
def histogram_stretching(location):

    def check_gray(img_path):
        img = cv2.imread(img_path)
        if img.ndim == 2:
            return True
        elif img.ndim == 3:
            w, h, _ = img.shape
            for i in range(w):
                for j in range(h):
                    r, g, b = img[i,j,:]
                    if r != g != b: 
                        return False
        return True
    def stretch_2D(img):
        hist,_ = np.histogram(img.flatten(),256,[0,256])
        hist =  hist.reshape(1,256)
        y = np.zeros((1,256))
        
        for i in range(256):
            if hist[0,i] == 0:
                y[0,i] = 0
            else:
                y[0,i] = i

        minv = np.min(y[y != 0])
        maxv = np.max(y[y != 0])
        #op img range 0-255
        stretch = np.round((((255 - 0)/(maxv - minv))*(y - minv))+0)
        stretch[stretch < 0] = 0
        stretch[stretch > 255] = 255
        for i in range(img.shape[0]):
                for j in range(img.shape[1]):
                    k = img[i,j]
                    img[i,j] = stretch[0,k]
        return img
    
    if check_gray(location):
        image = cv2.imread(location,0)
        output = stretch_2D(image)
        
    else:
        image = cv2.imread(location,1)
        output = np.zeros_like(image)
        for i in range(3):
            output[:,:,i] = stretch_2D(image[:,:,i])
            
    return output

In [10]:
def gamma_correction(location, gamma):
    img = cv2.imread(location)
    gamma_op = np.zeros_like(img)
    gamma_op =((img/ 255) ** gamma)*255
    return gamma_op

In [12]:
X  = "Lena.png"
original = cv2.imread(X)
gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
cv2.imwrite("Lena_gray.png", gray)

True

In [12]:
#Trial 2: Successful but referred https://github.com/omerferhatt/clahe-infrared-image-enhance/blob/master/clahe.py

# CLAHE
def adaptive_histogram(location, clipLimit = 2.0, tileGridSize = np.array([[8], [8]])):

    #check 2d and 3d image
    def check_grey(img_path):
        img = cv2.imread(img_path)
        if img.ndim == 2:
            return True
        elif img.ndim == 3:
            w, h, _ = img.shape
            for i in range(w):
                for j in range(h):
                    r, g, b = img[i,j,:]
                    if r != g != b: 
                        return False
        return True

    
    #Image interpolation function, need to be applied after CLAHE
    def interpolate_tile(subBin,lu,ru,lb,rb,sub_X,sub_Y):
        sub_img = np.zeros(subBin.shape)
        num = sub_X*sub_Y
        for i in range(sub_X):
            inverseI = sub_X-i
            for j in range(sub_Y):
                inverseJ = sub_Y-j
                val = subBin[i,j].astype(int)
                sub_img[i,j] = np.floor((inverseI*(inverseJ*lu[val] + j*ru[val])+ i*(inverseJ*lb[val] + j*rb[val]))/num)
        return sub_img


    #CLAHE FUNCTION
    def clahe(img,clip_Limit,tileGridSize):
        Total_bins = 256

        h,w = img.shape
        """
        if clip_Limit == 1:
            return
        """
        Total_Xtiles = tileGridSize[0,0]   #no. of tiles in X direction
        Total_Ytiles = tileGridSize[1,0]   #no. of tiles in Y direction
        xtile_size = np.round(h/ Total_Xtiles).astype(int)
        ytile_size = np.round(w/ Total_Ytiles).astype(int)
        tile_pixels = xtile_size*ytile_size
        
        # Excess number of pixels to get an integer value of Total_Xtiles and Total_Ytiles
        excess_X = int(xtile_size * (Total_Xtiles - h / xtile_size))
        excess_Y = int(ytile_size * (Total_Ytiles - w / ytile_size))
        
         # Pad that number of pixels to the image
        if excess_X != 0:
            img = np.append(img, np.zeros((excess_X, img.shape[1])).astype(int), axis=0)
        if excess_Y != 0:
            img = np.append(img, np.zeros((img.shape[0], excess_Y)).astype(int), axis=1)
            
        op_img = np.zeros(img.shape,dtype=img.dtype)
        
        if clip_Limit > 0:
            clip_Limit = max(1, clip_Limit*xtile_size*ytile_size/Total_bins)
        else:
            clip_Limit = 50
        
        #Lookup Table
        minVal = 0
        maxVal = 255
        
        binSz = np.floor(1+(maxVal-minVal)/Total_bins)
        LUT = np.floor((np.arange(minVal,maxVal+1)-minVal)/binSz)
        
        #Creating bins from LUT with image
        bins = LUT[img]
        #Making Histogram
    
        #print(Total_Xtiles,Total_Ytiles,hist.shape)
        hist = np.zeros((Total_Xtiles,Total_Ytiles,Total_bins))
        for i in range(Total_Xtiles):
            for j in range(Total_Ytiles):
                bin_ = bins[i*xtile_size:(i+1)*xtile_size,j*ytile_size:(j+1)*ytile_size].astype(int)
                for i1 in range(xtile_size):
                    for j1 in range(ytile_size):
                        hist[i,j,bin_[i1,j1]]+=1
        
        
        #Histogram Clipping
        if clip_Limit>0:
            for i in range(Total_Xtiles):
                for j in range(Total_Ytiles):
                    extra = 0
                    for nr in range(Total_bins):
                        excess = hist[i,j,nr] - clip_Limit # Calculate the total number of excess pixels.
                        if excess>0:
                            extra += excess
                            
                    # clip histogram and redistribute excess pixels in each bin 
                    incr_bin = extra/Total_bins       #avg bin increment
                    upper = clip_Limit - incr_bin  # bins larger than this set to clip_Limit
                    for nr in range(Total_bins):
                        if hist[i,j,nr] > clip_Limit:
                            hist[i,j,nr] = clip_Limit
                        else:
                            if hist[i,j,nr]>upper:
                                extra += upper - hist[i,j,nr]
                                hist[i,j,nr] = clip_Limit
                            else:
                                extra -= incr_bin
                                hist[i,j,nr] += incr_bin
                    
                    if extra > 0:
                        stepSz = max(1,np.floor(1+extra/Total_bins))
                        for nr in range(Total_bins):
                            extra -= stepSz
                            hist[i,j,nr] += stepSz
                            if extra < 1:
                                break
        
        #Histogram Mapping
        map_ = np.zeros((Total_Xtiles,Total_Ytiles,Total_bins))
        scale = (maxVal - minVal)/tile_pixels
        for i in range(Total_Xtiles):
            for j in range(Total_Ytiles):
                sum_ = 0
                for nr in range(Total_bins):
                    sum_ += hist[i,j,nr]
                    map_[i,j,nr] = np.floor(min(minVal+sum_*scale,maxVal))
    
        xI = 0
        for i in range(Total_Xtiles+1):   # top row
            if i==0:      
                sub_X = int(xtile_size/2)
                x_up = 0
                x_bottom = 0
            elif i==Total_Xtiles:        # bottom row
                sub_X = int(xtile_size/2)
                x_up = Total_Xtiles-1
                x_bottom = Total_Xtiles-1
            else:
                sub_X = xtile_size
                x_up = i-1
                x_bottom = i
            
            yI = 0
            for j in range(Total_Ytiles+1):      # left column 
                if j==0:
                    sub_Y = int(ytile_size/2)
                    y_left = 0
                    y_right = 0
                elif j==Total_Ytiles:            # right column
                    sub_Y = int(ytile_size/2)
                    y_left = Total_Ytiles-1
                    y_right = Total_Ytiles-1
                else:
                    sub_Y = ytile_size
                    y_left = j-1
                    y_right = j
            #corner points ul=upper-left ur=upper-right bl=bottom-left br=bottom-right
                ul = map_[x_up,y_left,:]
                ur = map_[x_up,y_right,:]
                bl = map_[x_bottom,y_left,:]
                br = map_[x_bottom,y_right,:]
                subBin = bins[xI:xI+sub_X,yI:yI+sub_Y]
                sub_img = interpolate_tile(subBin,ul,ur,bl,br,sub_X,sub_Y)
                op_img[xI:xI+sub_X,yI:yI+sub_Y] = sub_img
                yI += sub_Y
            xI += sub_X
      
      
        if excess_X == 0 and excess_Y != 0:
            return op_img[:, :-excess_Y]
        elif excess_X != 0 and excess_Y == 0:
            return op_img[:-excess_X, :]
        elif excess_X != 0 and excess_Y != 0:
            return op_img[:-excess_X, :-excess_Y]
        else:
            return op_img


    if check_grey(location):
        image_gray = cv2.imread(location,0) 
        output = np.zeros_like(image_gray)
        output = clahe(image_gray, clipLimit, tileGridSize)

    else:
        image = cv2.imread(location,1)
        output = np.zeros_like(image)
        for i in range(3):
            output[:,:,i] = clahe(image[:,:,i], clipLimit, tileGridSize)        
    return output


In [13]:
#colab code on jupyter notebook test

def histogram_equalization(input_image):

  def check_grey_scale(img_path):
    img = cv2.imread(img_path)
    w, h, c= img.shape
    for i in range(w):
        for j in range(h):
            r, g, b = img[i,j,:]
            if r != g != b: 
                return False
    return True

  def hist_equalize(oldimg):
    newimg=np.ones(oldimg.shape,dtype=np.int8)*-1
    oldhist=cv2.calcHist([oldimg],[0],None,[256],[0,256])
    oldhist=oldhist/oldhist.sum()
    oldhist=np.cumsum(oldhist) * (len(oldhist)-1)
    oldhist=np.round_(oldhist)
    oldhist=oldhist.astype(np.uint8)
    dict1={}
    for i in range(len(oldhist)):
      dict1[i]=oldhist[i]
    for key,val in dict1.items():
      newimg[oldimg==key]=val
    return newimg.astype(np.uint8)

  #for grey img
  if check_grey_scale(input_image) == True:
    greyimg=cv2.imread(input_image,0)
    output=hist_equalize(greyimg)

  #for colour img
  if check_grey_scale(input_image) == False:
    RGBimg=cv2.imread(input_image,1)
    Bimg= RGBimg[:,:,0]
    Gimg= RGBimg[:,:,1]
    Rimg= RGBimg[:,:,2]
    Boutput=hist_equalize(Bimg)
    Goutput=hist_equalize(Gimg)
    Routput=hist_equalize(Rimg)
    output=np.dstack((Boutput,Goutput,Routput))

  return output


ip=cv2.imread('Lena.png',0)
op=histogram_equalisation('Lena.png')

# cv2_imshow(ip)
# cv2_imshow(op)
iphist=cv2.calcHist([ip],[0],None,[256],[0,256])
ophist=cv2.calcHist([op],[0],None,[256],[0,256])
# fig, axs=plt.subplots(2)
# axs[0].plot(iphist)
# axs[1].plot(ophist)
plt.plot(ophist)
plt.show()


#INBUIT FUNC TO CHECK HISTOGRAM EQUALIZATION
equ = cv2.equalizeHist(ip)
ophist=cv2.calcHist([equ],[0],None,[256],[0,256])
plt.plot(ophist)
plt.show()

NameError: name 'histogram_equalisation' is not defined

In [14]:
# Trial 3: Successful with bilinear interpolation but shows tiles in img

def adaptive_histogram(location, cliplimit = 40, titlegridsize = np.array([[8], [8]])):
    
    #check 2d and 3d image
    def check_grey(img_path):
        img = cv2.imread(img_path)
        if img.ndim == 2:
            return True
        elif img.ndim == 3:
            w, h, _ = img.shape
            for i in range(w):
                for j in range(h):
                    r, g, b = img[i,j,:]
                    if r != g != b: 
                        return False
        return True
   
    
    def clip_tile(hist_img, Clip):
        Excess_pix = 0    
        # Calculate the total number of excess pixels.
        for k in range (hist_img.size):
            extra = hist_img[k] - Clip
            if extra > 0:
                Excess_pix += extra
        # clip histogram and redistribute excess pixels in each bin 
        avg_incr = Excess_pix / hist_img.size  #avg bin increment
        upper_limit = Clip - avg_incr   # bins larger than this set to cliplimit 
        for i in range(hist_img.size):
            if hist_img[i] > Clip:
                hist_img[i] = Clip
            else:
                if hist_img[i] > upper_limit :
                    Excess_pix += upper_limit - hist_img[i]
                    hist_img[i] = Clip
                else:
                    Excess_pix -= avg_incr
                    hist_img[i] += avg_incr
        # if 0 < excess_pixcels < avg_incr
        if Excess_pix > 0:
            for i in range(hist_img.size):
                if hist_img[i] != Clip:
                    hist_img[i] += 1
                    Excess_pix -= 1
                    if Excess_pix < 1:
                        break
        return hist_img
    
    def interpolation_tile(img , tile_size):
        img_inter = img.astype(np.int16)
        for i in range (tile_size[1,0] , img.shape[1] ,tile_size[1,0] ):
            for j in range (img.shape[0]):
                x= img_inter[ j, i+2]
                y = img_inter[j , i-3]
                img_inter[j,i] = (x+y)/2
                img_inter[j,i-1] = (x+y)/2
                img_inter[j,i-2] = (x+y)/2
                img_inter[j-1,i+1] = (x+y)/2
                
        for i in range (tile_size[0,0] , img.shape[0] ,tile_size[0,0] ):
            for j in range (img.shape[1]):
                x= img_inter[j+2, i]
                y = img_inter[j-3, i]
                img_inter[j,i] = (x+y)/2
                img_inter[j-1,i] = (x+y)/2
                img_inter[j+1,i] = (x+y)/2
                img_inter[j-2,i] = (x+y)/2
        a = img_inter.astype(int)
        return(a)
        
    
    def clahe_tile(img, cl):
        img1 = img.flatten()
        hist = np.zeros(256)
        for i in img1:
            hist[i] = hist[i]+1
            
        h_new = clip_tile(hist, cl)
        pdf = h_new/ len(img1)
        cdf = np.around(np.cumsum(pdf)*255)
        temp = []
        for pixel in img1:
            temp.append(cdf[pixel])
            
        img2 = np.reshape(temp,(img.shape[0],img.shape[1]))
        op_img = interpolation_tile(img2, tilesize)
        
        return(op_img)
    
    
    if check_grey(location):
        image_gray = cv2.imread(location,0)
        tilesize = np.array([[0], [0]])
        tilesize[0,0] = np.ceil(image_gray.shape[0]/titlegridsize[0,0])
        tilesize[1,0] = np.ceil(image_gray.shape[1]/titlegridsize[1,0])
        output = np.zeros_like(image_gray)
        for i in range(0,image_gray.shape[0],tilesize[0,0]):
            for j in range (0,image_gray.shape[1],tilesize[1,0]):
                output[i:i+tilesize[0,0],j:j+tilesize[1,0]] = clahe_tile(image_gray[i:i+tilesize[0,0],j:j+tilesize[1,0] ], cliplimit)
            
        return(output)
                   
        
    else:
        image = cv2.imread(location,1)
        tilesize = np.array([[0], [0]])
        tilesize[0,0] = np.ceil(image.shape[0]/titlegridsize[0,0])
        tilesize[1,0] = np.ceil(image.shape[1]/titlegridsize[1,0])
        output = np.zeros_like(image)
        for k in range(3):
            for i in range(0,image.shape[0],tilesize[0,0]):
                for j in range (0,image.shape[1],tilesize[1,0]):
                    output[i:i+tilesize[0,0],j:j+tilesize[1,0],k] = clahe_tile(image[i:i+tilesize[0,0],j:j+tilesize[1,0] ,k], cliplimit)
            
        return(output)

In [15]:
X  = "clahe.jpg"
ahe_op =adaptive_histogram(X, 40, np.array([[8], [8]]) )
cv2.imshow("CLAHE", ahe_op)
cv2.waitKey(0)
cv2.destroyAllWindows()